pip install nest_asyncio

git clone --depth=1 https://github.com/twintproject/twint.git
cd twint
pip3 install . -r requirements.txt

+ aiohttp==3.7.0 setzen in requirements.txt

In [1]:
import twint
import pandas as pd
import datetime
import schedule
import time
import os
import nest_asyncio
nest_asyncio.apply()
pd.set_option('display.max_columns', None)

c = twint.Config()

In [2]:
def searchTwint(keyword, since=None, until=None, language=None, limit=None, createDataFrame=True, hide_out=True):
    c = twint.Config()
    c.Search = keyword
    c.Filter_retweets = False
    if language is not None:
        c.Lang = language
    if since is not None:
        c.Since = since
    if until is not None:
        c.Until = until
    if limit is not None:
        c.Limit = limit
    c.Hide_output = hide_out
    if createDataFrame:
        c.Pandas = True
    twint.run.Search(c)
    if createDataFrame:
        Tweets_df = twint.storage.panda.Tweets_df
        return Tweets_df

In [24]:
df = searchTwint("#drachenlord", since="2022-05-18", until="2022-05-19", language=None, limit=100, createDataFrame=True)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [25]:
df.head(4)

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,urls,photos,video,thumbnail,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1527044605887320064,1527044605887320064,1.652911e+12,2022-05-18 23:52:59,+0200,,#Drachenlord Rainer hat verdächtig lange nich...,de,[drachenlord],[],1519805494222856193,1519805494222856193,HairedTroll,Red Haired Troll,3,23,https://twitter.com/HairedTroll/status/1527044...,[],[],0,,False,14,4,2,,#drachenlord,,,,,,,[],,,,
1,1527041272564830221,1526991958027149312,1.652910e+12,2022-05-18 23:39:45,+0200,,"@fraukaschkohler Wenn nicht er, wer dann? Mich...",de,[drachenlord],[],4891325434,4891325434,UMS907,UMS,3,23,https://twitter.com/UMS907/status/152704127256...,[],[],0,,False,3,0,0,,#drachenlord,,,,,,,"[{'screen_name': 'fraukaschkohler', 'name': 'F...",,,,
2,1527022277086306305,1527022277086306305,1.652905e+12,2022-05-18 22:24:16,+0200,,#drachenlord #drachenarmy https://t.co/0fE93...,und,"[drachenlord, drachenarmy]",[],1357061133421707265,1357061133421707265,kpop_haider,Luani,3,22,https://twitter.com/kpop_haider/status/1527022...,[],[https://pbs.twimg.com/media/FTESqx4XwAEp5AB.jpg],1,https://pbs.twimg.com/media/FTESqx4XwAEp5AB.jpg,False,17,12,1,,#drachenlord,,,,,,,[],,,,
3,1527019338628317189,1527019338628317189,1.652905e+12,2022-05-18 22:12:35,+0200,,Und schwupps die nächste Plattform auf die der...,de,[drachenlord],[],1464337747754332168,1464337747754332168,SolarSpaceAMore,SolarSpaceAMore,3,22,https://twitter.com/SolarSpaceAMore/status/152...,[],[],0,,False,3,0,1,https://twitter.com/DracheSchaut/status/152699...,#drachenlord,,,,,,,[],,,,


In [26]:
# Duplikate werden entfernt
df = df.drop_duplicates(subset=["id"])

In [27]:
df["date"] = df["date"].astype("datetime64")
#df["created_at"] = df["created_at"].astype("datetime64")
df = df[df["language"] == "de"]
df["Wochentag"] = df["date"].apply(lambda x: x.weekday())
#df["Stunde"] = df["time"].apply(lambda x: x.hour)
df["Originale_Tweetlaenge"] = df["tweet"].apply(lambda x: len(x))

In [28]:
df

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,urls,photos,video,thumbnail,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,Wochentag,Originale_Tweetlaenge
0,1527044605887320064,1527044605887320064,1.652911e+12,2022-05-18 23:52:59,+0200,,#Drachenlord Rainer hat verdächtig lange nich...,de,[drachenlord],[],1519805494222856193,1519805494222856193,HairedTroll,Red Haired Troll,3,23,https://twitter.com/HairedTroll/status/1527044...,[],[],0,,False,14,4,2,,#drachenlord,,,,,,,[],,,,,2,252
1,1527041272564830221,1526991958027149312,1.652910e+12,2022-05-18 23:39:45,+0200,,"@fraukaschkohler Wenn nicht er, wer dann? Mich...",de,[drachenlord],[],4891325434,4891325434,UMS907,UMS,3,23,https://twitter.com/UMS907/status/152704127256...,[],[],0,,False,3,0,0,,#drachenlord,,,,,,,"[{'screen_name': 'fraukaschkohler', 'name': 'F...",,,,,2,267
3,1527019338628317189,1527019338628317189,1.652905e+12,2022-05-18 22:12:35,+0200,,Und schwupps die nächste Plattform auf die der...,de,[drachenlord],[],1464337747754332168,1464337747754332168,SolarSpaceAMore,SolarSpaceAMore,3,22,https://twitter.com/SolarSpaceAMore/status/152...,[],[],0,,False,3,0,1,https://twitter.com/DracheSchaut/status/152699...,#drachenlord,,,,,,,[],,,,,2,96
4,1527007715075186689,1527007715075186689,1.652902e+12,2022-05-18 21:26:24,+0200,,#Drachenlord: Die Haider treiben mich dazu mit...,de,[drachenlord],[],1464337747754332168,1464337747754332168,SolarSpaceAMore,SolarSpaceAMore,3,21,https://twitter.com/SolarSpaceAMore/status/152...,[],[https://pbs.twimg.com/media/FTEFWQnXwAIVecV.jpg],1,https://pbs.twimg.com/media/FTEFWQnXwAIVecV.jpg,False,1,4,1,,#drachenlord,,,,,,,[],,,,,2,103
5,1526997203914465281,1526997203914465281,1.652899e+12,2022-05-18 20:44:38,+0200,,"Eine Seggstsucht ohne seggs, tadzächlich soga....",de,[drachenlord],[],1502341383117582344,1502341383117582344,BoynthahoodASB8,Boynthahood,3,20,https://twitter.com/BoynthahoodASB8/status/152...,[],[],0,,False,3,4,0,https://twitter.com/CathaleyaE/status/15269797...,#drachenlord,,,,,,,[],,,,,2,59
6,1526996579504345088,1526996579504345088,1.652899e+12,2022-05-18 20:42:09,+0200,,Was rendert der sich da zurecht? Utopisch. #dr...,de,[drachenlord],[],1502341383117582344,1502341383117582344,BoynthahoodASB8,Boynthahood,3,20,https://twitter.com/BoynthahoodASB8/status/152...,[],[],0,,False,8,2,0,https://twitter.com/Rainersgummipu1/status/152...,#drachenlord,,,,,,,[],,,,,2,55
7,1526988899498745860,1526988899498745860,1.652897e+12,2022-05-18 20:11:38,+0200,,#Drachenlord War grad am Essne Reinrendern und...,de,[drachenlord],[],1198637638133190657,1198637638133190657,SarkanyWolf,SarkanyWolf,3,20,https://twitter.com/SarkanyWolf/status/1526988...,[],[https://pbs.twimg.com/media/FTD0TPhXsAktKGM.jpg],1,https://pbs.twimg.com/media/FTD0TPhXsAktKGM.jpg,False,0,0,1,,#drachenlord,,,,,,,[],,,,,2,216
8,1526982431395434499,1526982431395434499,1.652896e+12,2022-05-18 19:45:56,+0200,,Ich kann nicht mehr! 🤣🤣🤣 Wie kann man sich nur...,de,[drachenlord],[],1458171961654325251,1458171961654325251,XFuckOffAndDieX,Pimmelberger Klopse,3,19,https://twitter.com/XFuckOffAndDieX/status/152...,[https://dracheporno.blogspot.com/2022/05/geil...,[],0,,False,4,0,0,,#drachenlord,,,,,,,[],,,,,2,244
9,1526981775012003841,1526981775012003841,1.652896e+12,2022-05-18 19:43:19,+0200,,Wenn beim #Drachenlord jetzt die Pron-offensiv...,de,"[drachenlord, liepe]",[],3246107308,3246107308,Coinboss_de,Coinboss,3,19,https://twitter.com/Coinboss_de/status/1526981...,[https://www.youtube.com/watch?v=qMl8Z8qvQQE],[],0,,False,0,0,0,,#drachenlord,,,,,,,[],,,,,2,129
10,1526981428554215431,1526981428554215431,1.652896e+12,2022-05-18 19:41:57,+0200,,Das frage ich mich auch....leider ist der IQ d...,de,"[drachenlord, kreiswichsverbot]",[],1404890581285933058,1404890581285933058,v2_der,Shlomo Ephraim Rosenschanzi Goldberg 🇮🇱🇺